In [3]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers.utils import load_image, make_image_grid
import numpy as np
import torch
from PIL import Image
import cv2

In [5]:
def get_inputs(prompt, images, batch_size=1):
    generator = [torch.Generator("cuda").manual_seed(i) for i in range(batch_size)]
    prompts = batch_size * [prompt]
    num_inference_steps = 20
    return {"prompt": prompts, "generator": generator, "num_inference_steps": num_inference_steps, "image":images}

In [ ]:


controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", use_safetensors=True)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", 
    controlnet=controlnet, 
    use_safetensors=True,
    safety_checker = None,
    requires_safety_checker = False).to("cuda")

original_image = load_image("https://huggingface.co/takuma104/controlnet_dev/resolve/main/bird_512x512.png")
original_image = original_image.resize((512, 512))
image = np.array(original_image)

low_threshold = 100
high_threshold = 200

image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)



batch_size = 16

canny_image_batch = np.array([image for i in range(batch_size)])



# image = pipe("", image=canny_image, guess_mode=True, guidance_scale=3.0).images[0]
augmented_images = pipe(**get_inputs("robin,high quality", canny_image_batch, batch_size)).images
make_image_grid([original_image, canny_image,], rows=1, cols=2)
make_image_grid(augmented_images, rows=2, cols=batch_size // 2)

Loading pipeline components...:  17%|█▋        | 1/6 [00:00<00:00,  6.29it/s]c:\Users\Austi\anaconda3\envs\pytorch_gpu\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 17.55it/s]
c:\Users\Austi\anaconda3\envs\pytorch_gpu\lib\site-packages\transformers\models\clip\modeling_clip.py:480: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  0%|          | 0/20 [00:00<?, ?it/s]